In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-28 13:44:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-28 13:44:53 (6.51 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Covid19").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [36]:
from pyspark import SparkFiles
url = "https://etlfinalproject.s3.amazonaws.com/published/PUBLIC/COVID-19-Activity/1614481307/COVID-19+Activity.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("charset", "utf-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)

df.show()

+---------------------------+-----------+-------------------+-----------+--------------+----------------+----------------------+------------------+--------------------+------------------+--------------------+-------------------------------+------------------+
|PEOPLE_POSITIVE_CASES_COUNT|COUNTY_NAME|PROVINCE_STATE_NAME|REPORT_DATE|CONTINENT_NAME|DATA_SOURCE_NAME|PEOPLE_DEATH_NEW_COUNT|COUNTY_FIPS_NUMBER|COUNTRY_ALPHA_3_CODE|COUNTRY_SHORT_NAME|COUNTRY_ALPHA_2_CODE|PEOPLE_POSITIVE_NEW_CASES_COUNT|PEOPLE_DEATH_COUNT|
+---------------------------+-----------+-------------------+-----------+--------------+----------------+----------------------+------------------+--------------------+------------------+--------------------+-------------------------------+------------------+
|                          0|   Cherokee|            Alabama| 2020-01-21|       America|  New York Times|                     0|              1019|                 USA|     United States|                  US|            

In [37]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(10)

+---------------------------+-----------+-------------------+-----------+--------------+----------------+----------------------+------------------+--------------------+------------------+--------------------+-------------------------------+------------------+
|PEOPLE_POSITIVE_CASES_COUNT|COUNTY_NAME|PROVINCE_STATE_NAME|REPORT_DATE|CONTINENT_NAME|DATA_SOURCE_NAME|PEOPLE_DEATH_NEW_COUNT|COUNTY_FIPS_NUMBER|COUNTRY_ALPHA_3_CODE|COUNTRY_SHORT_NAME|COUNTRY_ALPHA_2_CODE|PEOPLE_POSITIVE_NEW_CASES_COUNT|PEOPLE_DEATH_COUNT|
+---------------------------+-----------+-------------------+-----------+--------------+----------------+----------------------+------------------+--------------------+------------------+--------------------+-------------------------------+------------------+
|                          0|   Cherokee|            Alabama| 2020-01-21|       America|  New York Times|                     0|              1019|                 USA|     United States|                  US|            

In [83]:
# Create the death count by continent
people_death_df = df.groupby("CONTINENT_NAME").agg({"PEOPLE_DEATH_COUNT":"count"}).withColumnRenamed("count(CONTINENT_NAME)", "PEOPLE_DEATH_COUNT")
people_death_df.show(10)

+--------------+-------------------------+
|CONTINENT_NAME|count(PEOPLE_DEATH_COUNT)|
+--------------+-------------------------+
|        Europe|                    21359|
|        Africa|                    22568|
|          null|                     1209|
|       Oceania|                     6851|
|       America|                  1346020|
|          Asia|                    30225|
+--------------+-------------------------+



In [90]:
# Create the continent_by_province_table DataFrame and drop duplicates. 
continent_by_province_df = df.select(["CONTINENT_NAME"]).drop_duplicates()
continent_by_province_df.show(10)

+--------------+
|CONTINENT_NAME|
+--------------+
|        Europe|
|        Africa|
|          null|
|       Oceania|
|       America|
|          Asia|
+--------------+



In [64]:
# Create the covid cases by continent DataFrame. 
# Convert the 'REPORT_DATE' column to a date datatype with to_date("REPORT_DATE", 'yyyy-MM-dd').alias("REPORT_DATE")
date_cases_by_cont_df = df.select(["CONTINENT_NAME", "COUNTRY_SHORT_NAME", "PROVINCE_STATE_NAME", "COUNTY_NAME", to_date("REPORT_DATE", 'yyyy-MM-dd').alias("REPORT_DATE")])
date_cases_by_cont_df.show(10)

+--------------+------------------+-------------------+-----------+-----------+
|CONTINENT_NAME|COUNTRY_SHORT_NAME|PROVINCE_STATE_NAME|COUNTY_NAME|REPORT_DATE|
+--------------+------------------+-------------------+-----------+-----------+
|       America|     United States|            Alabama|   Cherokee| 2020-01-21|
|       America|     United States|            Alabama|   Cherokee| 2020-01-22|
|       America|     United States|            Alabama|   Cherokee| 2020-01-23|
|       America|     United States|            Alabama|   Cherokee| 2020-01-24|
|       America|     United States|            Alabama|   Cherokee| 2020-01-25|
|       America|     United States|            Alabama|   Cherokee| 2020-01-26|
|       America|     United States|            Alabama|   Cherokee| 2020-01-27|
|       America|     United States|            Alabama|   Cherokee| 2020-01-28|
|       America|     United States|            Alabama|   Cherokee| 2020-01-29|
|       America|     United States|     

In [65]:
# Create the Covid Continent table. DataFrame
covid_continent_table_df = df.select(["CONTINENT_NAME", "PEOPLE_POSITIVE_NEW_CASES_COUNT", "PEOPLE_POSITIVE_CASES_COUNT", "PEOPLE_DEATH_NEW_COUNT", "PEOPLE_DEATH_COUNT"])
covid_continent_table_df.show(10)

+--------------+-------------------------------+---------------------------+----------------------+------------------+
|CONTINENT_NAME|PEOPLE_POSITIVE_NEW_CASES_COUNT|PEOPLE_POSITIVE_CASES_COUNT|PEOPLE_DEATH_NEW_COUNT|PEOPLE_DEATH_COUNT|
+--------------+-------------------------------+---------------------------+----------------------+------------------+
|       America|                              0|                          0|                     0|                 0|
|       America|                              0|                          0|                     0|                 0|
|       America|                              0|                          0|                     0|                 0|
|       America|                              0|                          0|                     0|                 0|
|       America|                              0|                          0|                     0|                 0|
|       America|                              0|

In [77]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://final-project.czcueweyrsg8.us-west-2.rds.amazonaws.com:5432/Covid"
config = {"user":"postgres", 
          "password": "finalproject", 
          "driver":"org.postgresql.Driver"}

In [78]:

# Write people_death_df to table in RDS
people_death_df.write.jdbc(url=jdbc_url, table='people_death_table', mode=mode, properties=config)

In [79]:
# Write continent_by_province_df to table in RDS
continent_by_province_df.write.jdbc(url=jdbc_url, table='continent_table', mode=mode, properties=config)

In [80]:
# Write date_cases_by_cont_df to table in RDS
date_cases_by_cont_df.write.jdbc(url=jdbc_url, table='date_cases_table', mode=mode, properties=config)

In [81]:
# Write covid_continent_table_df to table in RDS
covid_continent_table_df.write.jdbc(url=jdbc_url, table='covid_continent_table', mode=mode, properties=config)